# Modeling {stage}_{target}
Using the feautures of the stage {stage} predict the {target} (target)

-------
**INFO MODEL**
- **stage**: d0eop
- **target**: microkappa

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Optimization-Industrial-Process


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [11]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import gcsfs
import pickle

from sklearn.model_selection import train_test_split

# models
from sklearn.linear_model import LinearRegression # lr
from sklearn.linear_model import Ridge # ridge
from sklearn.linear_model import Lasso # lasso
from sklearn.tree import DecisionTreeRegressor # tree
from sklearn.ensemble import GradientBoostingRegressor #gb
from sklearn.ensemble import RandomForestRegressor #rf
from xgboost import XGBRegressor # xgb
from  sklearn.neural_network import MLPRegressor # mlp

In [4]:
### desarrollo

PROJECT_ID = PROJECT_GCP
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### 1. Read data

In [5]:
path_data = 'artifacts/data/data.pkl'
data = pd.read_pickle(path_data)
data.head()

,230AIT446.PNT,240AIC022.MEAS,240AIC126.MEAS,240AIC224.MEAS,240AIC286.MEAS,240AIC324.MEAS,240AIC433.MEAS,240AIT063A.PNT,240AIT063B.PNT,240AIT225A.PNT,...,S240ALDP022,S240ALDP031,S240ALDP032,S276PER002,S2MAQUINAT07,S76ALE017,SSTRIPPING015,calc_prod_d0,calc_prod_d1,calc_prod_p
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:05:00,11.555040,2.983948,11.346645,4.413519,4.352375,10.441675,4.292521,5.869320,62.37495,1.837519,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3240.8635,3313.6215,3259.3745
2021-01-01 00:10:00,11.552320,3.015669,11.353215,4.413179,4.347186,10.432170,4.289684,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3260.7475,3301.6920,3208.6785
2021-01-01 00:15:00,11.549955,3.018903,11.355525,4.408321,4.355828,10.410115,4.284427,5.869320,62.37495,1.814020,...,91.49,1.8,11.4,11.77,1.5712,173.6,964.0,3265.5765,3284.1330,3210.7790
2021-01-01 00:20:00,11.547145,3.001164,11.326725,4.408659,4.361292,10.379145,4.285478,5.835750,62.37495,1.814020,...,91.49,1.7,11.3,11.77,1.5712,173.6,964.0,3253.7750,3271.9260,3221.7745
2021-01-01 00:25:00,11.543160,3.017393,11.336345,4.408596,4.356374,10.387205,4.304148,5.802179,62.37495,1.814020,...,91.49,1.6,11.2,11.77,1.5712,173.6,964.0,3236.9790,3267.3050,3227.6935


### 2. Read master tags data for this stage

In [9]:
stage = 'd0eop'
path_maestro_tags_d0eop = f'config/config_models/MaestroTags-{stage}-general.xlsx'
maestro_tags = pd.read_excel(path_maestro_tags_d0eop)
#maestro_tags

### 3. Define target
All the stages could have multiple targets. But with the models supported by gurobi only accept models that predict one target

In [7]:
list_target = ['XXXX']

### 4. Define features
In the exploratory data analysis you should select the features that will be use to predict the target. In this example, this features are defined manually in a list

In [8]:
list_features = []

In [10]:
list_features_controlables = []